In [ ]:
! pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import pandas as pd
import numpy as np
import wandb
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import Callback

# Initialize wandb
wandb.init(project="text_classification_project", config={
    "epochs": 10,
    "batch_size": 32,
    "max_sequence_length": 100,
    "embedding_input_dim": 50000,
    "embedding_output_dim": 300,
    "gru_units": 256,
    "dense_units": 700,
    "num_classes": 9248,  # Update this based on your dataset
    "learning_rate": "adam",
})

config = wandb.config

np.random.seed(2022)

# Data loading
data = pd.read_csv(r"/content/new_augmented_file_methode1_reduit.csv")

# Data preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['RawText'])
sequences = tokenizer.texts_to_sequences(data['RawText'])
sequences = pad_sequences(sequences, maxlen=config.max_sequence_length)

# Encoding labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data[' ICD10'])

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Model building
model = Sequential()
model.add(Embedding(input_dim=config.embedding_input_dim, output_dim=config.embedding_output_dim, input_length=config.max_sequence_length))
model.add(Bidirectional(GRU(config.gru_units)))
model.add(Dense(config.dense_units, activation='relu'))
model.add(Dense(config.num_classes, activation='softmax'))

# Model compilation
model.compile(loss='sparse_categorical_crossentropy', optimizer=config.learning_rate, metrics=['accuracy'])

# Wandb Logging Callback
class WandbLoggingCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        wandb.log({'epoch': epoch, 'loss': logs.get('loss'), 'val_loss': logs.get('val_loss'), 'accuracy': logs.get('accuracy'), 'val_accuracy': logs.get('val_accuracy')})

# Model training
model.fit(X_train, y_train, epochs=config.epochs, batch_size=config.batch_size, validation_data=(X_test, y_test), callbacks=[WandbLoggingCallback()])

# Finish the wandb run
wandb.finish()

Epoch 1/10
9252/9252 [==============================] - 207s 22ms/step - loss: 1.4156 - accuracy: 0.7156 - val_loss: 0.3882 - val_accuracy: 0.8767
Epoch 2/10
9252/9252 [==============================] - 197s 21ms/step - loss: 0.3197 - accuracy: 0.8929 - val_loss: 0.2964 - val_accuracy: 0.8998
Epoch 3/10
9252/9252 [==============================] - 203s 22ms/step - loss: 0.2591 - accuracy: 0.9070 - val_loss: 0.2706 - val_accuracy: 0.9060
Epoch 4/10
9252/9252 [==============================] - 197s 21ms/step - loss: 0.2330 - accuracy: 0.9128 - val_loss: 0.2575 - val_accuracy: 0.9096
Epoch 5/10
9252/9252 [==============================] - 196s 21ms/step - loss: 0.2225 - accuracy: 0.9153 - val_loss: 0.2452 - val_accuracy: 0.9112
Epoch 6/10
9252/9252 [==============================] - 195s 21ms/step - loss: 0.2221 - accuracy: 0.9159 - val_loss: 0.2436 - val_accuracy: 0.9143
Epoch 7/10
9252/9252 [==============================] - 201s 22ms/step - loss: 0.2252 - accuracy: 0.9154 - val_loss: 0

accuracy,▁▇████████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▂▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇█████
val_loss,█▄▂▂▁▁▂▂▂▂
accuracy,0.91646
epoch,9
loss,0.22176
val_accuracy,0.91318
val_loss,0.27374


In [ ]:
model.save("LeModel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
X_test.shape

(74015, 100)

In [ ]:
y_test.shape

(74015,)

In [ ]:
# Si vos étiquettes sont encodées, assurez-vous qu'elles sont dans le bon format pour l'évaluation
loss, accuracy = model.evaluate(X_test, y_test)
print("Perte sur l'ensemble de test:", loss)
print("Précision sur l'ensemble de test:", accuracy)

2313/2313 [==============================] - 18s 8ms/step - loss: 0.2737 - accuracy: 0.9132
Perte sur l'ensemble de test: 0.27374187111854553
Précision sur l'ensemble de test: 0.9131797552108765


In [ ]:
def predire_code_icd():
    # Récupérer le texte d'entrée de l'utilisateur
    texte_entrée = input("Entrez votre texte ici : ")

    # Procéder à la tokenisation et au padding
    sequence_entrée = tokenizer.texts_to_sequences([texte_entrée])
    sequence_padded = pad_sequences(sequence_entrée, maxlen=100)  # Assurez-vous que cela correspond à la longueur utilisée lors de l'entraînement

    # Faire une prédiction avec le modèle
    prediction = model.predict(sequence_padded)

    # Obtenir l'indice de la classe la plus probable
    indice_classe_predite = np.argmax(prediction, axis=-1)

    # Convertir l'indice en code ICD
    code_icd_predite = label_encoder.inverse_transform(indice_classe_predite)[0]

    # Retourner ou imprimer le code ICD prédit
    print(f"Code ICD prédit: {code_icd_predite}")

# Appeler la fonction pour faire une prédiction basée sur l'entrée de l'utilisateur
predire_code_icd()